In [ ]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    tqdm \
    pinecone-notebooks==0.1.1 \
    python-dotenv

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
EMBED_DIM = 768

# Create index 
from pipecone_connection import PineConeDB
# Initialize Pinecone

pc = PineConeDB(PINECONE_API_KEY)
hasaki_index = pc.create_index(index_name='hasaki-index', dimension=EMBED_DIM)


## Upsert campaign-index (Updated)

In [2]:
campaigns = pd.read_csv('../../../data/results/campaigns.csv')
campaigns.rename(columns={'Unnamed: 0': 'campaignID'}, inplace=True)
campaigns.head()

,campaignID,label,link,img
0,0,"Khai trương chi nhánh 221, 222 - Cần Thơ, Đồng...",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240-172...
1,1,"Khai trương chi nhánh 223, 224 - Hội An, TP.HCM",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240mobi...
2,2,Sale lương về - Deal ngập tràn,https://hasaki.vn/campaign/best-seller,https://media.hcdn.vn/hsk/campaign/640x240---2...
3,3,[HOT] Ưu đãi trải nghiệm làm đẹp đến hơn 60%,https://hasaki.vn/campaign/trai-nghiem-dich-vu,https://media.hcdn.vn/hsk/campaign/wap16981146...
4,4,Unilever Nâng Niu Nét Đẹp Toàn Diện,https://hasaki.vn/campaign/unilever-khuyen-mai...,https://media.hcdn.vn/hsk/campaign/UnileverSIS...


In [3]:
from pipecone_connection import create_vector_emb

campaigns['vector'] = campaigns['label'].apply(lambda x: create_vector_emb(x))
campaigns.head()

/home/phatpham/anaconda3/envs/rag-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,campaignID,label,link,img,vector
0,0,"Khai trương chi nhánh 221, 222 - Cần Thơ, Đồng...",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240-172...,"[0.09607130289077759, 0.31030184030532837, -0...."
1,1,"Khai trương chi nhánh 223, 224 - Hội An, TP.HCM",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240mobi...,"[-0.19367045164108276, -0.004278192762285471, ..."
2,2,Sale lương về - Deal ngập tràn,https://hasaki.vn/campaign/best-seller,https://media.hcdn.vn/hsk/campaign/640x240---2...,"[0.031419891864061356, 0.08881331980228424, -0..."
3,3,[HOT] Ưu đãi trải nghiệm làm đẹp đến hơn 60%,https://hasaki.vn/campaign/trai-nghiem-dich-vu,https://media.hcdn.vn/hsk/campaign/wap16981146...,"[0.16664496064186096, 0.32210487127304077, -0...."
4,4,Unilever Nâng Niu Nét Đẹp Toàn Diện,https://hasaki.vn/campaign/unilever-khuyen-mai...,https://media.hcdn.vn/hsk/campaign/UnileverSIS...,"[-0.10489341616630554, 0.008368105627596378, -..."


In [6]:
# Prepare and upsert data
campaigns_list = [{"id": str(row['campaignID']), 
         "values": row['vector'],
         "metadata": {'label': row['label'], 'link': row['link'], 'img': row['img']}} for _, row in campaigns.iterrows()]
hasaki_index.upsert(vectors=campaigns_list,
                    namespace='campaign-namespace')


{'upserted_count': 13}

In [ ]:
# ref: https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/gen-qa-openai.ipynb#scrollTo=KPXw1sEubwoF

## Upsert support-indexn (Updated)

In [7]:
support = pd.read_csv('../../../data/results/supports.csv')
# support = support.drop(columns=['Unnamed: 0'], axis=1)
support.rename(columns={'Unnamed: 0': 'supportID'}, inplace=True)

support.head()

,supportID,title,link,content
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...


In [9]:
from pipecone_connection import create_vector_emb

support['vector'] = support['content'].apply(lambda x: create_vector_emb(x))
support.head()

,supportID,title,link,content,vector
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,"[0.01456688717007637, 0.33099138736724854, -0...."
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...,"[0.07909272611141205, 0.3365171551704407, -0.3..."
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...,"[0.07966456562280655, 0.3297780156135559, -0.4..."
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...,"[0.035692255944013596, 0.3778902292251587, -0...."
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...,"[0.02337840385735035, 0.2968924045562744, -0.3..."


In [14]:
import re
# .apply(lambda x: re.sub(r'<[^>]+>', '', x))
# Prepare and upsert data
support_list = [{"id": str(row['supportID']), 
         "values": row['vector'],
         "metadata": {'title': re.sub(r'<[^>]+>', '', row['title']), 'link': row['link'], 'content': row['content']}} for _, row in support.iterrows()]

hasaki_index.upsert(vectors=support_list,
                    namespace = 'support-namespace')

{'upserted_count': 27}

## Upsert category-index (Updated)

In [16]:
categories = pd.read_csv('../../../data/results/categories.csv')
categories.rename(columns={'Unnamed: 0': 'categoryID'}, inplace=True)
categories.head()

,categoryID,link,img,cat
0,0,https://hasaki.vn/danh-muc/trang-diem-moi-c24....,https://media.hcdn.vn/catalog/category/c24-tra...,Trang Điểm Môi
1,1,https://hasaki.vn/danh-muc/mat-na-c30.html,https://media.hcdn.vn/catalog/category/30_1_im...,Mặt Nạ
2,2,https://hasaki.vn/danh-muc/trang-diem-mat-c52....,https://media.hcdn.vn/catalog/category/c52-tra...,Trang Điểm Mặt
3,3,https://hasaki.vn/danh-muc/sua-rua-mat-c19.html,https://media.hcdn.vn/catalog/category/19_3_im...,Sữa Rửa Mặt
4,4,https://hasaki.vn/danh-muc/trang-diem-vung-mat...,https://media.hcdn.vn/catalog/category/c50-tra...,Trang Điểm Mắt


In [17]:
from pipecone_connection import create_vector_emb
categories['vector'] = categories['cat'].apply(lambda x: create_vector_emb(x))
categories.head()

,categoryID,link,img,cat,vector
0,0,https://hasaki.vn/danh-muc/trang-diem-moi-c24....,https://media.hcdn.vn/catalog/category/c24-tra...,Trang Điểm Môi,"[0.04586426913738251, -0.06818058341741562, 0...."
1,1,https://hasaki.vn/danh-muc/mat-na-c30.html,https://media.hcdn.vn/catalog/category/30_1_im...,Mặt Nạ,"[-0.05931249260902405, 0.08366265147924423, -0..."
2,2,https://hasaki.vn/danh-muc/trang-diem-mat-c52....,https://media.hcdn.vn/catalog/category/c52-tra...,Trang Điểm Mặt,"[-0.016898442059755325, -0.3402135968208313, -..."
3,3,https://hasaki.vn/danh-muc/sua-rua-mat-c19.html,https://media.hcdn.vn/catalog/category/19_3_im...,Sữa Rửa Mặt,"[-0.11072025448083878, -0.45872682332992554, -..."
4,4,https://hasaki.vn/danh-muc/trang-diem-vung-mat...,https://media.hcdn.vn/catalog/category/c50-tra...,Trang Điểm Mắt,"[-0.3483051359653473, -0.42001697421073914, -0..."


In [18]:
# Prepare and upsert data
category_list = [{"id": str(row['categoryID']), 
         "values": row['vector'],
         "metadata": {'link': row['link'], 'img': row['img'], 'cat': row['cat']}} for _, row in categories.iterrows()]
hasaki_index.upsert(vectors=category_list,
              namespace="category-namespace")


{'upserted_count': 15}

## Upsert product-pname-index

In [85]:
import pandas as pd
product_data = pd.read_csv('../../../data/results/product_data_v2.csv')
product_data.drop(columns={'Unnamed: 0'}, inplace=True)
product_data.shape

(2868, 20)

In [86]:
product_data = product_data.drop(columns=['price_extended', 'about', 'combo'])
product_data.columns

Index(['pid', 'plink', 'pname', 'pcat', 'price', 'delivery', 'meta',
       'ingredients', 'usage', 'img', 'desc', 'count_NEG', 'count_NEU',
       'count_POS', 'cmt_summary_NEG', 'cmt_summary_NEU', 'cmt_summary_POS'],
      dtype='object')

In [87]:
product_data = product_data.drop_duplicates(subset=['pid'])
product_data.shape

(2811, 17)

In [88]:
product_data['price'] = product_data['price'].apply(lambda x: int(re.sub(r'\D', '', x)))
product_data['delivery'] = product_data['delivery'].apply(lambda x: x.replace('\n', ' '))

In [89]:
product_data.head()

,pid,plink,pname,pcat,price,delivery,meta,ingredients,usage,img,desc,count_NEG,count_NEU,count_POS,cmt_summary_NEG,cmt_summary_NEU,cmt_summary_POS
0,232100001,https://hasaki.vn/san-pham/son-duong-moi-khong...,Son Dưỡng Môi DHC Không Màu Hỗ Trợ Giảm Thâm M...,Trang Điểm / Trang Điểm Môi / Son Dưỡng Môi,159000,Miễn phí vận chuyển Giao Nhanh Miễn Phí 2H tại...,Barcode4511413503997Thương HiệuDHCXuất xứ thươ...,Thành phần sản phẩmThành phần chính:Dầu ôliu:G...,Hướng dẫn sử dụngThoa nhẹ nhàng son dưỡng DHC ...,['https://media.hasaki.vn/wysiwyg/HaNguyen1/so...,Son Dưỡng DHC Lip Creamlà một trong những dòng...,1.0,0.0,13.0,Mùi k thấy thơm mà ngai ngái kiểu đồ rởm là sa...,NaN,Cây này cây thứ 3 rồi vẫn chưa có dấu hiệu dừn...
1,100540031,https://hasaki.vn/san-pham/sap-duong-moi-hong-...,Sáp Dưỡng Môi Vaseline Hồng Xinh 7g,Trang Điểm / Trang Điểm Môi / Son Dưỡng Môi,65000,Miễn phí vận chuyển Giao Nhanh Miễn Phí 2H tại...,Barcode305210231597Thương HiệuVaselineXuất xứ ...,Thành phần sản phẩmNội dung đang được cập nhật,Hướng dẫn sử dụngKhi môi các bạn bi khô nên sử...,['https://media.hcdn.vn/wysiwyg/Chau/son-duong...,Đôi môi khô của bạn sẽ mềm mịn hẳn nếu sử dụng...,2.0,1.0,10.0,sáp dưỡng này thoa tối thi môi mềm mịn nhưng s...,d,tẩy tế bào chét môi tốt lắm á tốt cực kì nhỏ g...
2,422209457,https://hasaki.vn/san-pham/son-kem-li-black-ro...,Son Kem Lì Black Rouge A12 Dashed Brown Nâu Gạ...,Trang Điểm / Trang Điểm Môi / Son Kem / Tint,164000,Miễn phí vận chuyển Giao Nhanh Miễn Phí 2H tại...,Barcode8809464341559Thương HiệuBlack RougeXuất...,Thành phần sản phẩm1. A09 Sweet CinnamonThành ...,Hướng dẫn sử dụngDùng cọ lấy một lượng son vừa...,['https://media.hcdn.vn/wysiwyg/Chau/son-kem-l...,Son Kem Lì Black Rouge Air Fit Velvet Version ...,0.0,1.0,4.0,NaN,Màu đẹp nhưng chất son rất rất khô môi luôn,"lên màu siêu đẹp luôn mọi người, hàng chính hã..."
3,100830002,https://hasaki.vn/san-pham/nuoc-tay-trang-mat-...,Nước Tẩy Trang Mắt Môi Maybelline 150ml,Trang Điểm / Trang Điểm Môi / Tẩy Trang Mắt / Môi,112000,Miễn phí vận chuyển Giao Nhanh Miễn Phí 2H tại...,Barcode6902395843467Thương HiệuMaybellineXuất ...,Thành phần sản phẩmThành phần chính:Panthenol:...,Hướng dẫn sử dụngLắc đều trước khi sử dụng.Thấ...,['https://media.hcdn.vn/wysiwyg/Chau/nuoc-tay-...,Nước Tẩy Trang Mắt Môi Maybelline Lip & Eyes M...,0.0,3.0,5.0,NaN,shop cho em hỏi nếu dùng tẩy môi mắt rồi có cầ...,"Quá ok Loại này tẩy trang mascara vs môi đỉnh,..."
4,422209392,https://hasaki.vn/san-pham/son-thoi-3ce-vo-tro...,Son Thỏi 3CE Vỏ Trong Suốt Unstained Red - Đỏ ...,Trang Điểm / Trang Điểm Môi / Son Thỏi,294000,Miễn phí vận chuyển Giao Nhanh Miễn Phí 2H tại...,Barcode8809664984488Thương Hiệu3CEXuất xứ thươ...,"Thành phần sản phẩmDimethicone, Isoamyl Laurat...","Hướng dẫn sử dụngTrước khi thoa son, nên tẩy t...",['https://media.hcdn.vn/wysiwyg/HaNguyen1/son-...,Son Thỏi 3CE Vỏ Trong Suốt Soft Matte Lipstick...,0.0,0.0,10.0,NaN,NaN,Màu son đẹp và lì❤️ Dùng son này mềm nôi và mà...


In [90]:
product_data.to_csv('../../../data/results/preprocessed_product.csv', index=False)

In [71]:
from pipecone_connection import create_vector_emb
product_data['vector'] = product_data['pname'].apply(lambda x: create_vector_emb(x))
product_data['desc_vector'] = product_data['desc'].apply(lambda x: create_vector_emb(x))

In [72]:
product_data.to_csv('vectorized_product.csv', index=False)

In [77]:
# Prepare and upsert data
product_list = [{"id": str(row['pid']), 
         "values": row['vector'],
         "metadata": {'product_id': row['pid'], 'pname': row['pname']}} for _, row in product_data.iterrows()]

# Define a batch size 
batch_size = 100  

for i in range(0, len(product_list), batch_size):
    batch = product_list[i:i + batch_size]
    hasaki_index.upsert(vectors=batch,
                        namespace='product-pname-namespace')

## Upsert product-desc-index (Updated)

In [84]:
# Prepare and upsert data
product_list = [{"id": str(row['pid']), 
         "values": row['desc_vector'],
         "metadata": {'product_id': row['pid'], 'pname': row['pname']}} for _, row in product_data.iterrows()]

batch_size = 100  

for i in range(0, len(product_list), batch_size):
    batch = product_list[i:i + batch_size]
    hasaki_index.upsert(vectors=batch,
                        namespace='product-desc-namespace')